# Objectif

- Prédire les émissions de CO2 (TotalGHGEmissions)
- Prédire la consommation totale d'énérgie (SiteEnergyUseWN(kBtu) ou SiteEnergyUse(kBtu))
- Prendre en compte uniquement les bâtiments non destinés à l'habitation
- Evaluer l'intérêt de la feature "ENERGYSTARScore" car fastidieux à calculer

L'énoncé nous précise que ces données sont compliquées à obtenir d'où l'intérêt de créer un modèle d'IA pouvant prédire les données futurs à partir d'un échantillon de 2016

## Targets

**Nos targets sont donc** :
- TotalGHGEmissions
- SiteEnergyUse(kBtu) ou SiteEnergyUseWN(kBtu)

# Exploratory Data Analysis

## Analyse rapide

- 3376 lignes, 46 colonnes
- 27 valeurs discrètes, 12 valeurs catégorielles

## Suppressions

- OSEBuildingId ne nous est d'aucune utilité
- À travers la colonne "Outlier". On utilise cette donnée dès le début car sinon les traitements futurs vont la retirer ("Outlier" possède 99% de `NaN`)
- On garde les features qui ont assez de données. Une feature avec une majorité de `NaN` ne nous sera d'aucune utilité
    - 3 features ont un ratio > 90% de valeurs non définies
    - 4 features ont 1 voir 0 types de données
- Sélectionner les lignes qui respectent le prérequis des bâtiments non destinées à l'habitation
    - Le type de bâtiment (BuildingType)
    - La colonne "ComplianceStatus"
    - La colonne "PrimaryPropertyType"
- On supprime le peu d'outliers qui n'ont pas été détectés avec la colonne Outlier.

Après ce traitement, on se rend compte qu'on a une majorité de valeurs définies pour plein de features. Vu qu'il nous reste 1300 lignes après conservation des données suivant les critères ci-dessus, on voir données manquantes sont représentées en lignes (sur le graphique) et au vu du peu de données manquantes sur les colonnes, on peut tout simplement supprimer ces lignes car trop laborieux de les traiter pour une faible plus-value.

**Post traitement** :

- 1327 lignes, 40 colonnes
- **DefaultData** : pourrait avoir un impacte négatif. En effet, d'après la documentation sur cette variable elle signifie, quand vraie, qu'une colonne a été remplie à partir de valeurs par défaut. Cette valeur aurait pu être importante pour nos prédictions. On pourra l'analyser dans une étape d'optimisation.
    - Après traitement, cette valeur ne possède que des valeurs False. Elle n'est donc plus utile au projet
- **ComplianceStatus** : Valeur "Compliant" uniquement. On peut donc retirer cette feature

## Analyse

### Corrélations

- Les variables standardes et WN/bBtu... sont très corrélées
    - Peu d'impacte sur les mêmes données avec des grandeurs différentes (qui possèdent malgré tout des fluctuations aussi petites soient-elles)
- SourceEUI et SiteEUI très corrélé
- EnergyUse et Electricity très corrélé

### Distribution

- **ENERGYSTARScore**
    - Une majorité de valeurs élevées
- **Les 2 targets**
    - Liens à vérifier entre TotalGHGEmissions et
        - NumberofBuildings
        - NumberofFloors
        - PropertyGFATotal
        - PropertyGFAParking
        - PropertyGFABuilding(s)
        - LargestPropertyUseTypeGFA
        - SiteEnergyUse(kBtu)
        - SiteEnergyUseWN(kBtu)
        - SteamUse(kBtu)
        - Electricity(kWh)
        - Electricity(kBtu)
        - NaturalGas(therms)
        - NaturalGas(kBtu)
        - TotalGHGEmissions
- **Des données par type**

# Pre-processing

# Modeling    

# Exploratory Data Analysis

## Analyse rapide

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as ps
import seaborn as sns
import math

pd.options.display.max_columns = 50
np.set_printoptions(linewidth=180)
sns.set_style("dark")
%matplotlib inline

# Servira pour la partie analyse
target_cols = ["TotalGHGEmissions", "SiteEnergyUseWN(kBtu)"]

# Analyse & Traitement rapide

In [ ]:
df = pd.read_csv("2016_Building_Energy_Benchmarking.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

On regarde les colonnes et leur type

In [ ]:
df.dtypes.value_counts()

In [ ]:
df = df.drop(columns="OSEBuildingID")

Outliers sera supprimé plus tard dû au nombre de valeurs `NaN`. Profitons-en et analysons le peu de valeurs disponibles

In [ ]:
df.Outlier.unique()

In [ ]:
df = df[df.Outlier.isin(values=[np.nan])]
df.Outlier.unique()

On regarde le nombre de nulls

In [ ]:
def nulls(df):
    plt.figure(figsize=(20,8))
    plt.title("Répartition des valeurs non définies sous forme de heatmap")
    g = sns.heatmap(df.isna(), cbar=False)
    g.set_xticklabels(df.columns, rotation=45, ha="right")
    plt.plot()
    
nulls(df)

In [ ]:
(df.isna().sum() / df.shape[0]).sort_values(ascending=False)

In [ ]:
df.nunique().sort_values()

In [ ]:
df = df[df.columns[(((df.isna().sum() / df.shape[0]) < 0.9) & (df.nunique() >= 2))]]
df

On peut regarder quelles valeurs on a avec certaines colonnes

In [ ]:
df.ComplianceStatus.unique()

In [ ]:
df.CouncilDistrictCode.unique()

In [ ]:
df.BuildingType.unique()

In [ ]:
df.PrimaryPropertyType.unique()

In [ ]:
df.LargestPropertyUseType.unique()

On veut conserver uniquement les lignes utiles et les bâtiments non destinés à l'habitation

On peut utiliser :
- Le type de bâtiment
- La colonne "ComplianceStatus"
- La colonne "PrimaryPropertyType"

In [ ]:
df = df[~df.BuildingType.isin(values=["Multifamily MR (5-9)", "Multifamily LR (1-4)", "Multifamily HR (10+)"])]
df.BuildingType.unique()

In [ ]:
df = df[~df.ComplianceStatus.isin(values=["Missing Data", "Error - Correct Default Data"])]
df.ComplianceStatus.unique()

In [ ]:
df = df[~df.PrimaryPropertyType.isin(values=["Hotel", "Mixed Use Property", "Residence Hall", "Low-Rise Multifamily"])]
df.PrimaryPropertyType.unique()

In [ ]:
df = df[~df.LargestPropertyUseType.isin(values=["Other - Lodging/Residential", "Residential Care Facility"])]
df.LargestPropertyUseType.unique()

In [ ]:
nulls(df)

On remarque qu'il a beaucoup de features pour lesquelles les données sont toutes définies.

Il y a seulement quelques données pour les features : LargestPropertyUseTypeGFA ; SiteEUI(kBtu/sf) ; SiteEnergyUseWN(kBtu) ... dont les données sont manquantes.

On peut utiliser les 3 features mentionnées pour repérer les `NaN` et les retirer car elles ne représentent pas une très grande proportion du dataset, donc pas besoin d'imputer par la moyenne ou la médiane.

In [ ]:
df[["LargestPropertyUseTypeGFA", "SiteEUI(kBtu/sf)", "SiteEnergyUseWN(kBtu)"]].isna().sum()

In [ ]:
(df["LargestPropertyUseTypeGFA"].isna() | df["SiteEUI(kBtu/sf)"].isna() | df["SiteEnergyUseWN(kBtu)"].isna()).value_counts()

In [ ]:
df = df[~(df["LargestPropertyUseTypeGFA"].isna() | df["SiteEUI(kBtu/sf)"].isna() | df["SiteEnergyUseWN(kBtu)"].isna())]
nulls(df)

In [ ]:
df.drop(columns=["ENERGYSTARScore", "SecondLargestPropertyUseType", "SecondLargestPropertyUseTypeGFA", "ThirdLargestPropertyUseType","ThirdLargestPropertyUseTypeGFA", "ZipCode"]).isna().sum().sum()

Hormis les colonne ci-dessus, on a que des valeurs définies

In [ ]:
df.shape

In [ ]:
cols = ["SiteEnergyUseWN(kBtu)", "SiteEnergyUse(kBtu)"]

In [ ]:
for o in df.select_dtypes("object"):
    print(f"{o:-<32} {df[o].unique()[:5]}")

In [ ]:
px.scatter(df[cols], x=cols[0], y=cols[1])

In [ ]:
m = df[cols[1]].median()
for i, (siteEnergyWN, siteEnergy) in enumerate(zip(df[cols[0]], df[cols[1]])):
    if(siteEnergy - siteEnergyWN > m):
        df.loc[df.index[i], cols[0]] =  df.loc[df.index[i], cols[1]]

In [ ]:
px.scatter(df[cols], x=cols[0], y=cols[1])

In [ ]:
df.Neighborhood.unique()

In [ ]:
df.Neighborhood.replace({"North": "NORTH", "Central": "CENTRAL", "Northwest": "NORTHWEST",
                        "Delridge": "DELRIDGE", "Ballard": "BALLARD",
                        "DELRIDGE NEIGHBORHOODS": "DELRIDGE"}, 
                        inplace=True)
df.Neighborhood.unique()

In [ ]:
len(df.Neighborhood.unique())

## Analyse

In [ ]:
def corr():
    df[cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(df.corr())
corr()

In [ ]:
df.DefaultData.value_counts()

In [ ]:
df = df.drop(columns=["DefaultData"])
corr()

In [ ]:
for o in df.select_dtypes("object"):
    s = df[o].unique().size
    if s > 20:
        print(f"{o} contains {s} different values")
    else:
        plt.figure()
        df[o].value_counts().plot.pie()

In [ ]:
df.ComplianceStatus.unique()

In [ ]:
df = df.drop(columns="ComplianceStatus")

In [ ]:
plt.title("ENERGYSTARScore distribution")
sns.histplot(df.ENERGYSTARScore, kde=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

ns = df.select_dtypes(include=np.number)
ns[ns.columns] = StandardScaler().fit_transform(ns)

In [ ]:
ns.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

def distrib_by(df, col):
    lines = math.ceil((len(df.columns) - 1) / 5)
    fig, axs = plt.subplots(lines, 5, figsize=(18,int(lines * 3)), constrained_layout=True)
    fig.suptitle(f"{col} distribution comparison")
    for i,n in enumerate(ns.drop(columns=col[0])):
        ax = axs[i // 5, i % 5]
        sns.histplot(df[[col[0], n]], kde=True, ax=ax)
        ax.set_xlim(-3, 3)
        ax.set_ylim(0, 150)
        ax.set_title(f"{n}")

distrib_by(ns, target_cols[0])

In [ ]:
df["TotalGHGEmissions"].isna().sum()

In [ ]:
df["GHGEmissionsIntensity"].isna().sum()

In [ ]:
cols = ["TotalGHGEmissions", "GHGEmissionsIntensity"]
df[cols] = df[cols].dropna()

In [ ]:
px.scatter(df[cols], x=cols[0], y=cols[1])

In [ ]:
df[df.TotalGHGEmissions > 4000]

In [ ]:
def bivariate_analysis(cols):
    df_analysis = df[cols]
    lines = math.ceil((len(df_analysis.columns) - 1) / 4)
    for c1 in df_analysis.columns:
        fig, axs = plt.subplots(lines, 4, figsize=(18,int(lines * 3)), constrained_layout=True)
        fig.suptitle("Bivariate analysis of %s" % c1)
        for i, c2 in enumerate(df_analysis.drop(columns=c1).columns):
            ax = axs[int(i / 4), i % 4]
            
            ax.set_xlabel(c1)
            ax.set_ylabel(c2)
            data = df[[c1, c2]].dropna()
            ax.scatter(data[c1], data[c2])

bivariate_analysis(["YearBuilt", "NumberofBuildings", "NumberofFloors", "PropertyGFATotal", "PropertyGFAParking", "PropertyGFABuilding(s)", 
                "LargestPropertyUseTypeGFA", "SiteEUIWN(kBtu/sf)", "SourceEUIWN(kBtu/sf)", "SiteEnergyUseWN(kBtu)", "SteamUse(kBtu)",
                "Electricity(kBtu)", "NaturalGas(kBtu)", "TotalGHGEmissions", "GHGEmissionsIntensity", "ENERGYSTARScore"])

In [ ]:
df[df.NumberofBuildings > 10]

In [ ]:
unique_years = np.sort(df.YearBuilt.unique())
unique_years

In [ ]:
year_range_min = min(unique_years)
year_range_max = max(unique_years)
year_range_min_10 = int(int(year_range_min / 10) * 10)
year_range_max_10 = int(int(year_range_max / 10) * 10)
year_range_bins = range(year_range_min_10, year_range_max_10 + 20, 10)
year_range_labels = ["{}-{}".format(x, x+9) for x in range(year_range_min_10,year_range_max_10, 10)]
year_range_labels.append("2010+")

df["YearBuiltRange"] = pd.cut(df.YearBuilt,
                            bins=year_range_bins,
                            labels=year_range_labels)


df["YearBuiltRange"].value_counts().sort_index().plot(kind="bar")
df[["YearBuilt","YearBuiltRange"]].head()

In [ ]:
(df.PropertyGFAParking + df["PropertyGFABuilding(s)"] - df.PropertyGFATotal != 0).sum()

## Sélection & Encodage

In [ ]:
df

In [ ]:
df.select_dtypes(include=["int64", "float64"]).fillna(0, inplace=True)

In [ ]:
keep = ["YearBuiltRange", "Neighborhood", "NumberofBuildings", "NumberofFloors", "PropertyGFAParking",
        "PropertyGFABuilding(s)", "SiteEUIWN(kBtu/sf)", "SourceEUIWN(kBtu/sf)", 
        "SiteEnergyUseWN(kBtu)", "SteamUse(kBtu)", "Electricity(kBtu)", "NaturalGas(kBtu)", "TotalGHGEmissions", 
        "GHGEmissionsIntensity"]
df[keep].to_csv("cleaned.csv", index=False)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.99)

In [ ]:
from sklearn.preprocessing import StandardScaler

n = df[df.columns[df.isna().sum() == 0]].select_dtypes(include=np.number)
n = StandardScaler().fit_transform(n)
pca.fit(n)


In [ ]:
n.shape

In [ ]:
pca.n_components_

In [ ]:
pca = PCA(n_components=2)
pca.fit(n)
t = pca.transform(n)
sns.scatterplot(t)